In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re,json,nltk

In [ ]:
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
sub=pd.read_csv('/content/sample_submission.csv')

In [ ]:
train

In [ ]:
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()/train.shape[0]*100

In [ ]:
train.describe()

In [ ]:
# Replace [None] and 'N/A' with NaN
for col in train.columns:
    train[col] = train[col].replace({np.nan: None, '[None]': None, 'N/A': None}) # Remove ['N/A'] from replace

for col in test.columns:
    test[col] = test[col].replace({np.nan: None, '[None]': None, 'N/A': None}) # Remove ['N/A'] from replace

for col in sub.columns:
    sub[col] = sub[col].replace({np.nan: None, '[None]': None, 'N/A': None}) # Remove ['N/A'] from replace

train.info()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(train.isnull())

In [ ]:
null_percentage = train.isnull().sum() * 100 / len(train)
print(null_percentage)

In [ ]:
# Drop columns with high null percentage as indicated
columns_to_drop = ['extra_curricular_activity_types', 'extra_curricular_organization_names',
                   'extra_curricular_organization_links', 'role_positions', 'languages',
                   'proficiency_levels', 'certification_providers', 'certification_skills',
                   'online_links', 'issue_dates', 'expiry_dates', 'address']

train = train.drop(columns=columns_to_drop, errors='ignore')
test = test.drop(columns=columns_to_drop, errors='ignore')

# Display info of the modified DataFrames (optional)
train.info()
test.info()

In [ ]:
duplicate_rows = train[train.duplicated()]
print(duplicate_rows)

In [ ]:
train.drop_duplicates(inplace=True)

In [ ]:
train.shape

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk

try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

def clean_text(text):
    if pd.isna(text): #check if text is nan if so return it without changing
        return text
    text = re.sub(r'[^\w\s]', '', str(text)) #convert to string here
    text = re.sub(r'\[|\]', '', text)
    text = text.lower()
    words = text.split()
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    text = " ".join(words)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

for col in train.select_dtypes(include=['object']).columns:
    train[col] = train[col].apply(clean_text)

for col in test.select_dtypes(include=['object']).columns:
    test[col] = test[col].apply(clean_text)

In [ ]:
# prompt: apply steamming

from nltk.stem import PorterStemmer

try:
    PorterStemmer()
except LookupError:
    nltk.download('punkt')


stemmer = PorterStemmer()

def stem_text(text):
    if pd.isna(text):
        return text
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

for col in train.select_dtypes(include=['object']).columns:
    train[col] = train[col].apply(stem_text)

for col in test.select_dtypes(include=['object']).columns:
    test[col] = test[col].apply(stem_text)

In [ ]:
!pip install nltk

In [ ]:
 # Install the NLTK library
import nltk

# Download required resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets') # This downloads a list of available tagsets, including 'eng'
nltk.download('averaged_perceptron_tagger_eng') # This line is added to download 'averaged_perceptron_tagger_eng'

def add_tokens(text):
    if pd.isna(text):
        return text
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return " ".join([f"{word}/{tag}" for word, tag in pos_tags])

for col in train.select_dtypes(include=['object']).columns:
    train[col] = train[col].apply(add_tokens)

for col in test.select_dtypes(include=['object']).columns:
    test[col] = test[col].apply(add_tokens)

In [ ]:
# Install the NLTK library
import nltk

# Download required resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets') # This downloads a list of available tagsets, including 'eng'
nltk.download('averaged_perceptron_tagger_eng') # This line is added to download 'averaged_perceptron_tagger_eng'
nltk.download('punkt_tab') # Download the 'punkt_tab' data package

def add_tokens(text):
    if pd.isna(text):
        return text
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return " ".join([f"{word}/{tag}" for word, tag in pos_tags])

for col in train.select_dtypes(include=['object']).columns:
    train[col] = train[col].apply(add_tokens)

for col in test.select_dtypes(include=['object']).columns:
    test[col] = test[col].apply(add_tokens)

In [ ]:
train

In [ ]:
null_percentage = train.isnull().sum() * 100 / len(train)
print(null_percentage)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split

In [ ]:
# # Label Encoding with Null Value Preservation
# def label_encode_with_nulls(train_df, test_df, column):
#     le = LabelEncoder()

#     # Fit the encoder on the training data, handling nulls
#     train_values = train_df[column].dropna()
#     le.fit(train_values)

#     # Transform both train and test data, preserving nulls
#     train_df[column] = train_df[column].map(lambda x: le.transform([x])[0] if pd.notna(x) else x)
#     test_df[column] = test_df[column].map(lambda x: le.transform([x])[0] if pd.notna(x) else x)
#     return train_df, test_df

# # Apply label encoding to object columns, preserving nulls
# for col in train.select_dtypes(include=['object']).columns:
#     train, test = label_encode_with_nulls(train, test, col)

# # ... (rest of your code)

In [ ]:
# Label Encoding with Null Value Preservation
def label_encode_with_nulls(train_df, test_df, column):
    le = LabelEncoder()

    # Fit the encoder on the training data, handling nulls
    # Combine unique values from both train and test to ensure all labels are seen
    all_values = pd.concat([train_df[column], test_df[column]]).dropna().unique()
    le.fit(all_values) # Fit on combined unique values

    # Transform both train and test data, preserving nulls
    train_df[column] = train_df[column].map(lambda x: le.transform([x])[0] if pd.notna(x) else x)
    test_df[column] = test_df[column].map(lambda x: le.transform([x])[0] if pd.notna(x) else x)
    return train_df, test_df

# Apply label encoding to object columns, preserving nulls
for col in train.select_dtypes(include=['object']).columns:
    train, test = label_encode_with_nulls(train, test, col)

# ... (rest of your code)

In [ ]:
null_percentage = train.isnull().sum() * 100 / len(train)
print(null_percentage)

In [ ]:
train.describe()

In [ ]:
# # Fill missing values in train dataset with the mean of each column
# for col in train.columns:
#     if train[col].isnull().any():
#         mean_val = train[col].mean()
#         train[col].fillna(mean_val, inplace=True)

# # Fill missing values in test dataset with the mean of each column
# for col in test.columns:
#     if test[col].isnull().any():
#         mean_val = test[col].mean()
#         test[col].fillna(mean_val, inplace=True)

In [ ]:
# Fill missing values in train dataset with the mean of each column
for col in train.columns:
    if train[col].isnull().any():
        if pd.api.types.is_numeric_dtype(train[col]): # Check if column is numeric
            mean_val = train[col].mean()
            train[col].fillna(mean_val, inplace=True)
        else:
            # Handle non-numeric columns (e.g., fill with mode or a constant)
            mode_val = train[col].mode()[0]  # Get the mode
            train[col].fillna(mode_val, inplace=True)

# Fill missing values in test dataset with the mean of each column
for col in test.columns:
    if test[col].isnull().any():
        if pd.api.types.is_numeric_dtype(test[col]): # Check if column is numeric
            mean_val = test[col].mean()
            test[col].fillna(mean_val, inplace=True)
        else:
            # Handle non-numeric columns (e.g., fill with mode or a constant)
            mode_val = test[col].mode()[0]  # Get the mode
            test[col].fillna(mode_val, inplace=True)

In [ ]:
null_percentage = train.isnull().sum() * 100 / len(train)
print(null_percentage)

In [ ]:
test

In [ ]:
x=train.drop('matched_score', axis=1)
x

In [ ]:
y=train['matched_score']
y

In [ ]:
from sklearn.model_selection import train_test_split

# **Deep Learning**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# ANN

In [ ]:
# # ANN Model
# ann_model = Sequential([
#     Dense(64, input_dim=x_train.shape[1], activation='relu'),
#     Dropout(0.3),
#     Dense(32, activation='relu'),
#     Dropout(0.3),
#     Dense(y_train.shape[1], activation='softmax')  # Adjust for the number of classes
# ])

# ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# ann_model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout
from tensorflow.keras.utils import to_categorical

# ... (your existing code) ...

# One-hot encode y_train
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# ANN Model
ann_model = Sequential([
    Dense(64, input_dim=x_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')  # Now y_train.shape[1] will be the number of classes
])

ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
ann_model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# ... (rest of your code) ...

In [ ]:
from tensorflow.keras.utils import to_categorical

# ... (your existing code) ...

# One-hot encode y_train
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# ANN Model
ann_model = Sequential([
    Dense(64, input_dim=x_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')  # Now y_train.shape[1] will be the number of classes
])

ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
ann_model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# ... (rest of your code) ...

# RNN

In [ ]:
# Reshape input for RNN (3D input: samples, timesteps, features)
x_train_rnn = np.expand_dims(x_train, axis=1)
x_test_rnn = np.expand_dims(x_test, axis=1)

In [ ]:
# Reshape input for RNN (3D input: samples, timesteps, features)
X_train_rnn = np.expand_dims(x_train, axis=1)
X_test_rnn = np.expand_dims(x_test, axis=1)

rnn_model = Sequential([
    SimpleRNN(64, input_shape=(x_train_rnn.shape[1], x_train_rnn.shape[2]), activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')
])

rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
rnn_model.fit(x_train_rnn, y_train, epochs=20, batch_size=32, validation_split=0.2)


# LSTM

In [ ]:
# LSTM Model
lstm_model = Sequential([
    LSTM(64, input_shape=(x_train_rnn.shape[1], x_train_rnn.shape[2]), activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(x_train_rnn, y_train, epochs=20, batch_size=32, validation_split=0.2)


In [ ]:
# Evaluate ANN
ann_loss, ann_accuracy = ann_model.evaluate(x_test, y_test)
print(f"ANN Accuracy: {ann_accuracy:.2f}")

# Evaluate RNN
rnn_loss, rnn_accuracy = rnn_model.evaluate(x_test_rnn, y_test)
print(f"RNN Accuracy: {rnn_accuracy:.2f}")

# Evaluate LSTM
lstm_loss, lstm_accuracy = lstm_model.evaluate(x_test_rnn, y_test)
print(f"LSTM Accuracy: {lstm_accuracy:.2f}")


In [ ]:
# ANN
# Predictions for calculating MSE
from sklearn.metrics import mean_squared_error  # Import mean_squared_error
y_test_pred_ANN = ann_model.predict(x_test)

# Calculate MSE
mse_ANN = mean_squared_error(y_test, y_test_pred_ANN)
print(f"Mean Squared Error: {mse_ANN:.4f}")

In [ ]:
# ANN
# Predictions for calculating MSE
from sklearn.metrics import mean_squared_error  # Import mean_squared_error
y_train_pred_ANN = ann_model.predict(x_train)

# Calculate MSE
mse_ANN_train = mean_squared_error(y_train, y_train_pred_ANN)
print(f"Mean Squared Error: {mse_ANN_train:.4f}")

In [ ]:
# RNN
# Predictions for calculating MSE
y_train_RNN = rnn_model.predict(np.expand_dims(x_train, axis=1)) # Reshape x_train to add timesteps dimension

# Calculate MSE
mse_RNN = mean_squared_error(y_train, y_train_RNN)
print(f"Mean Squared Error: {mse_RNN:.4f}")

In [ ]:
# RNN
# Predictions for calculating MSE
y_test_RNN = rnn_model.predict(np.expand_dims(x_test, axis=1)) # Reshape x_train to add timesteps dimension

# Calculate MSE
mse_RNN_test = mean_squared_error(y_test, y_test_RNN)
print(f"Mean Squared Error: {mse_RNN_test:.4f}")

In [ ]:
# # LSTM
# # Predictions for calculating MSE
# y_train_pred = lstm_model.predict(x_train)

# # Calculate MSE
# mse = mean_squared_error(y_train, y_train_pred)
# print(f"Mean Squared Error: {mse:.4f}")

In [ ]:
# LSTM
# Predictions for calculating MSE
y_train_pred = lstm_model.predict(np.expand_dims(x_train, axis=1)) # Reshape x_train to add timesteps dimension

# Calculate MSE
mse = mean_squared_error(y_train, y_train_pred)
print(f"Mean Squared Error: {mse:.4f}")

In [ ]:
x_train

In [ ]:
test

# Submission File

In [ ]:
# Step 1: Extract and store the "ID" column from the "test" dataset
test_ids = test['ID']  # Store the "ID" column

In [ ]:
test_ids

In [ ]:
# Step 2: Drop the "ID" column from the "test" dataset to prepare the features
test = test.drop(columns=['ID'])  # Remove "ID" from the dataset

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer

try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

try:
    PorterStemmer()
except LookupError:
    nltk.download('punkt')

try:
    nltk.word_tokenize("test sentence")  # Check if 'punkt' is downloaded
except LookupError:
    nltk.download('punkt')

try:
    nltk.pos_tag(nltk.word_tokenize("test sentence"))  # Check if 'averaged_perceptron_tagger' is downloaded
except LookupError:
    nltk.download('averaged_perceptron_tagger')

try:
    nltk.help.upenn_tagset()  # Check if 'tagsets' is downloaded
except LookupError:
    nltk.download('tagsets')


def clean_text(text):
    if pd.isna(text):
        return text
    text = re.sub(r'[^\w\s]', '', str(text))
    text = re.sub(r'\[|\]', '', text)
    text = text.lower()
    words = text.split()
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    text = " ".join(words)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def stem_text(text):
    if pd.isna(text):
        return text
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

def add_tokens(text):
    if pd.isna(text):
        return text
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return " ".join([f"{word}/{tag}" for word, tag in pos_tags])

# Assuming 'train' and 'test' are your DataFrames
stemmer = PorterStemmer()
for col in train.select_dtypes(include=['object']).columns:
    train[col] = train[col].apply(clean_text)
    train[col] = train[col].apply(stem_text)
    train[col] = train[col].apply(add_tokens)

for col in test.select_dtypes(include=['object']).columns:
    test[col] = test[col].apply(clean_text)
    test[col] = test[col].apply(stem_text)
    test[col] = test[col].apply(add_tokens)


from sklearn.preprocessing import LabelEncoder
# Label Encoding with Null Value Preservation
def label_encode_with_nulls(train_df, test_df, column):
    le = LabelEncoder()

    # Fit the encoder on the training data, handling nulls
    # Combine unique values from both train and test to ensure all labels are seen
    all_values = pd.concat([train_df[column], test_df[column]]).dropna().unique()
    le.fit(all_values) # Fit on combined unique values

    # Transform both train and test data, preserving nulls
    train_df[column] = train_df[column].map(lambda x: le.transform([x])[0] if pd.notna(x) else x)
    test_df[column] = test_df[column].map(lambda x: le.transform([x])[0] if pd.notna(x) else x)
    return train_df, test_df

# Apply label encoding to object columns, preserving nulls
for col in train.select_dtypes(include=['object']).columns:
    train, test = label_encode_with_nulls(train, test, col)

# Continue with your model training and prediction

In [ ]:
# Install necessary library
!pip install scikit-learn

In [ ]:


import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# ... (your existing code for preprocessing) ...

# Label encode object columns in 'test' DataFrame
for col in test.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    test[col] = le.fit_transform(test[col].astype(str))  # Convert to string before encoding

# Convert 'test' DataFrame to numeric
test = test.apply(pd.to_numeric, errors='coerce')

# ... (rest of your code for model prediction) ...

# Predict on the test data using the trained LSTM model
test_rnn = np.expand_dims(test, axis=1)

# Convert to float32
test_rnn = test_rnn.astype(np.float32) # convert the data to float32
predictions = lstm_model.predict(test_rnn)

# ... (rest of your code for creating submission file) ...

In [ ]:
# # Save the submission file
# submission_df.to_csv('submission.csv', index=False)

In [ ]:
# Step 4: Create a submission DataFrame using the stored "ID" and predictions
submission_lstm = pd.DataFrame({
    'ID': test_ids,      # Use the stored "ID" column
    'matched_score': predictions[:, 1]  # Use the probabilities of the target class (class 1)
})



In [ ]:
sub

In [ ]:
submission_lstm

In [ ]:
# Step 5: Save the submission file to a CSV
submission_lstm.to_csv('submission_lstm.csv', index=False)

In [ ]:
# # prompt: same way predict for ANN

# predicted_classes = ann_model.predict(test)

# # Create a submission DataFrame
# submission_ANN = pd.DataFrame({'ID': test_ids, 'matched_score': predicted_classes})

# submission_ANN

In [ ]:
# prompt: same way predict for ANN

predicted_probs = ann_model.predict(test)  # Get predicted probabilities

# Assuming your predictions are probabilities, get the class with the highest probability
# or select the probability for the relevant class (e.g., class 1)
predicted_classes = predicted_probs[:, 1] # Change this index to 0 if it's binary classification

# Create a submission DataFrame
submission_ANN = pd.DataFrame({'ID': test_ids, 'matched_score': predicted_classes})

submission_ANN

In [ ]:
predicted_classes

In [ ]:
# prompt: Now for RNN

# Predict on the test data using the trained LSTM model
test_rnn = np.expand_dims(test, axis=1)

# Convert to float32
test_rnn = test_rnn.astype(np.float32)  # convert the data to float32
predictions = ann_model.predict(test)

# Assuming your predictions are probabilities, get the class with the highest probability
predicted_classes = np.argmax(predictions, axis=1)

# Create a submission DataFrame
submission_ANN = pd.DataFrame({'ID': test_ids, 'matched_score': y_test_pred_ANN[:1]})

submission_ANN
predicted_classes

In [ ]:
# Step 5: Save the submission file to a CSV
submission.to_csv('submission_lgbm_2.csv', index=False)

In [ ]:
y_test

In [ ]:
y_train

In [ ]:
ann = ann_model.predict(test)

In [ ]:
ann

In [ ]:
# Create a submission DataFrame
submission_ANN = pd.DataFrame({'ID': test_ids, 'matched_score': ann[:, 1]}) # Select the second column (index 1) of y_test_pred_ANN

submission_ANN